In [83]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2024, day=18)

def parses(data):
    walls = []
    for line in data.strip().split('\n'):
        x, y = line.split(',')
        walls.append((int(x), int(y)))
    return walls

# import re
# def parses(data):
#     return [[int(i) for i in re.findall("-?\d+", line)] 
#              for line in data.strip().split('\n')]

# def parses(data):
#     return data

data = parses(puzzle.input_data)

In [84]:
sample = parses("""5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0""")

In [85]:
# sample

In [86]:
from heapq import heappush, heappop

In [156]:
def shortest_path(data, N=70):
    walls = set(data)

    def heuristic(x,y):
        return abs(x-N) + abs(y-N)
        
    heap = [(heuristic(0,0),0,0,0)]
    visited = set()
    
    while heap:
        _, cost, x, y = heappop(heap)
        if (x, y) == (N, N):
            return cost
        if (x, y) in visited:
            continue
        visited.add((x,y))
        for dx, dy in [(1,0),(-1,0),(0,1),(0,-1)]:
            x2, y2 = x+dx, y+dy
            if (0 <= x2 <= N and
                0 <= y2 <= N and
                (x2,y2) not in walls and
                (x2,y2) not in visited):
                heappush(heap, (cost+1+heuristic(x2,y2), cost+1, x2, y2))        
    
def solve_a(data, N=70, steps=1024):
    return shortest_path(data[:steps], N=N)

In [157]:
solve_a(sample, N=6, steps=12)

22

In [158]:
solve_a(data, N=70)

374

In [146]:
def solve_b(data, N=70):
    a, b = 0, len(data)
    # binary search
    while b-a > 1:
        m = (a+b)//2
        
        s = shortest_path(data[:m], N=N)
        if s is None: # not feasible
            b = m
        else:
            a = m
    x, y = data[b-1] # [:b] has points 0...b-1
    return f'{x},{y}'

In [147]:
solve_b(sample, 6)

'6,1'

In [148]:
solve_b(data)

'30,12'

In [126]:
sample

[(5, 4),
 (4, 2),
 (4, 5),
 (3, 0),
 (2, 1),
 (6, 3),
 (2, 4),
 (1, 5),
 (0, 6),
 (3, 3),
 (2, 6),
 (5, 1),
 (1, 2),
 (5, 5),
 (2, 5),
 (6, 5),
 (1, 4),
 (0, 4),
 (6, 4),
 (1, 1),
 (6, 1),
 (1, 0),
 (0, 5),
 (1, 6),
 (2, 0)]

In [10]:
from collections import deque
import heapq

def parse_input(input_text):
    """Parse input text into list of coordinates."""
    coordinates = []
    for line in input_text.strip().split('\n'):
        if line:
            x, y = map(int, line.split(','))
            coordinates.append((x, y))
    return coordinates

def create_grid(size, corrupted):
    """Create grid with corrupted positions marked."""
    grid = [[False] * size for _ in range(size)]
    for x, y in corrupted:
        if 0 <= x < size and 0 <= y < size:
            grid[y][x] = True
    return grid

def find_shortest_path(grid):
    """Find shortest path from (0,0) to (target,target) using A* search."""
    size = len(grid)
    target = (size - 1, size - 1)
    
    def manhattan_distance(pos):
        return abs(pos[0] - target[0]) + abs(pos[1] - target[1])
    
    # Priority queue for A* search
    queue = [(manhattan_distance((0, 0)), 0, (0, 0))]  # (f_score, g_score, position)
    visited = set()
    
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # right, down, left, up
    
    while queue:
        f_score, g_score, current = heapq.heappop(queue)
        
        if current == target:
            return g_score
            
        if current in visited:
            continue
            
        visited.add(current)
        
        for dx, dy in directions:
            new_x = current[0] + dx
            new_y = current[1] + dy
            
            if (0 <= new_x < size and 0 <= new_y < size and 
                not grid[new_y][new_x] and 
                (new_x, new_y) not in visited):
                
                new_g = g_score + 1
                new_f = new_g + manhattan_distance((new_x, new_y))
                heapq.heappush(queue, (new_f, new_g, (new_x, new_y)))
    
    return None  # No path found

def solve_puzzle(input_text, grid_size=71, num_bytes=1024):
    """Main function to solve the puzzle."""
    coordinates = parse_input(input_text)
    # Take only first 1024 bytes
    corrupted = coordinates[:num_bytes]
    grid = create_grid(grid_size, corrupted)
    
    # Find shortest path
    shortest_path = find_shortest_path(grid)
    return shortest_path

# # Function to visualize grid (for debugging)
# def visualize_grid(grid):
#     for row in grid:
#         print(''.join('#' if cell else '.' for cell in row))

In [33]:
from collections import deque
from typing import List, Tuple, Set

def parse_input(input_text: str) -> List[Tuple[int, int]]:
    """Parse the input text into a list of (x,y) coordinates."""
    coordinates = []
    for line in input_text.strip().split('\n'):
        if line:
            x, y = map(int, line.split(','))
            coordinates.append((x, y))
    return coordinates

def get_corrupted_cells(coordinates: List[Tuple[int, int]], num_bytes: int) -> Set[Tuple[int, int]]:
    """Get the set of corrupted cells after num_bytes have fallen."""
    return set(coordinates[:num_bytes])

def is_valid_position(pos: Tuple[int, int], grid_size: int, corrupted: Set[Tuple[int, int]]) -> bool:
    """Check if a position is valid (within bounds and not corrupted)."""
    x, y = pos
    return (0 <= x <= grid_size and 
            0 <= y <= grid_size and 
            (x, y) not in corrupted)

def find_shortest_path(grid_size: int, corrupted: Set[Tuple[int, int]]) -> int:
    """Find shortest path from (0,0) to (grid_size,grid_size) avoiding corrupted cells."""
    if (0, 0) in corrupted or (grid_size, grid_size) in corrupted:
        return float('inf')
    
    start = (0, 0)
    end = (grid_size, grid_size)
    queue = deque([(start, 0)])  # (position, steps)
    visited = {start}
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    
    while queue:
        (x, y), steps = queue.popleft()
        
        if (x, y) == end:
            return steps
            
        for dx, dy in directions:
            next_pos = (x + dx, y + dy)
            if (next_pos not in visited and 
                is_valid_position(next_pos, grid_size, corrupted)):
                visited.add(next_pos)
                queue.append((next_pos, steps + 1))
    
    return float('inf')

def find_blocking_byte(input_text: str, grid_size: int = 70) -> Tuple[int, int]:
    """Find the first byte that blocks all paths to the exit."""
    coordinates = parse_input(input_text)
    
    # Try each byte one by one until we find one that blocks all paths
    for i in range(len(coordinates)):
        corrupted = get_corrupted_cells(coordinates, i + 1)
        path_length = find_shortest_path(grid_size, corrupted)
        
        if path_length == float('inf'):
            # This byte blocked all paths - return its coordinates
            return coordinates[i]
    
    return (-1, -1)  # No blocking byte found

def solve_part1(input_text: str, grid_size: int = 70) -> int:
    """Solve part 1 - find minimum steps needed."""
    coordinates = parse_input(input_text)
    
    # Binary search to find the minimum number of steps
    left = 0
    right = 1024  # First kilobyte
    best_steps = float('inf')
    
    while left <= right:
        mid = (left + right) // 2
        corrupted = get_corrupted_cells(coordinates, mid)
        steps = find_shortest_path(grid_size, corrupted)
        
        if steps < float('inf'):
            best_steps = min(best_steps, steps)
            right = mid - 1
        else:
            left = mid + 1
    
    return best_steps if best_steps < float('inf') else -1

def solve_part2(input_text: str, grid_size: int = 70) -> str:
    """Solve part 2 - find coordinates of first blocking byte."""
    x, y = find_blocking_byte(input_text, grid_size)
    return f"{x},{y}"

# Example usage:
example_input = """
5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0
"""

# Test both parts with example
print("Part 1:", solve_part1(puzzle.input_data, grid_size=70))  # Should print 22
print("Part 2:", solve_part2(puzzle.input_data, grid_size=70))  # Should print 6,1

Part 1: 140
Part 2: 30,12


In [12]:
solve_puzzle(puzzle.inputaa_data)

374

In [18]:
test_example()

Example result: 1,6


In [ ]:
def parse_coordinates(input_text):
    """Parse input text into list of coordinates."""
    coordinates = []
    for line in input_text.strip().split('\n'):
        if line.strip():  # Skip empty lines
            x, y = map(int, line.strip().split(','))
            coordinates.append((x, y))
    return coordinates

def is_path_possible(blocked_positions, size):
    """Check if path exists from (0,0) to (size-1,size-1) using BFS."""
    if (0, 0) in blocked_positions or (size-1, size-1) in blocked_positions:
        return False
        
    queue = [(0, 0)]
    visited = {(0, 0)}
    
    while queue:
        x, y = queue.pop(0)
        
        if (x, y) == (size-1, size-1):
            return True
            
        # Try all four directions
        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_x, new_y = x + dx, y + dy
            
            if (0 <= new_x < size and 
                0 <= new_y < size and 
                (new_x, new_y) not in blocked_positions and 
                (new_x, new_y) not in visited):
                visited.add((new_x, new_y))
                queue.append((new_x, new_y))
                
    return False

def find_blocking_coordinate(input_text, size=7):
    """Find first coordinate that makes path impossible."""
    coordinates = parse_coordinates(input_text)
    blocked = set()
    
    for i, coord in enumerate(coordinates):
        blocked.add(coord)
        if not is_path_possible(blocked, size):
            return coord
            
    return None

# Test with example
example = """5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0"""

def test():
    result = find_blocking_coordinate(example, 7)
    if result:
        print(f"Blocking coordinate: {result[0]},{result[1]}")
    else:
        print("No blocking coordinate found")

In [ ]:
def solve_a(data):
    pass

In [ ]:
solve_a(sample)

In [ ]:
solve_a(data)

In [ ]:
def solve_b(data):
    pass

In [ ]:
solve_b(sample)

In [ ]:
solve_b(data)

In [29]:
def parse_coordinates(input_text):
    """Parse input text into list of coordinates."""
    coordinates = []
    for line in input_text.strip().split('\n'):
        if line.strip():  # Skip empty lines
            x, y = map(int, line.strip().split(','))
            coordinates.append((x, y))
    return coordinates

def is_path_possible(blocked_positions, size):
    """Check if path exists from (0,0) to (size-1,size-1) using BFS."""
    if (0, 0) in blocked_positions or (size-1, size-1) in blocked_positions:
        return False
        
    queue = [(0, 0)]
    visited = {(0, 0)}
    
    while queue:
        x, y = queue.pop(0)
        
        if (x, y) == (size-1, size-1):
            return True
            
        # Try all four directions
        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_x, new_y = x + dx, y + dy
            
            if (0 <= new_x < size and 
                0 <= new_y < size and 
                (new_x, new_y) not in blocked_positions and 
                (new_x, new_y) not in visited):
                visited.add((new_x, new_y))
                queue.append((new_x, new_y))
                
    return False

def find_blocking_coordinate(input_text, size=7):
    """Find first coordinate that makes path impossible."""
    coordinates = parse_coordinates(input_text)
    blocked = set()
    
    for i, coord in enumerate(coordinates):
        blocked.add(coord)
        if not is_path_possible(blocked, size):
            return coord
            
    return None

# Test with example
example = """5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0"""

def test():
    result = find_blocking_coordinate(puzzle.input_data, 70)
    if result:
        print(f"Blocking coordinate: {result[0]},{result[1]}")
    else:
        print("No blocking coordinate found")

In [30]:
test()

Blocking coordinate: 69,69
